In [197]:
import csv
import statistics
from pprint import pprint

In [198]:
STUDENTS_DATA_FILE = "files/students_record.csv"
REPORT_FILE = "files/school_report.txt"

In [199]:
def load_student_data(filename):
    """
    Load student data from a CSV file into a list of dictionary items.


    Clean and prepare data values for analysis.
    """

    students = []

    with open(filename, "r") as file:
        reader = csv.DictReader(file)

        for row in reader:
            for key in row:
                # Convert values from string to integer
                if key in ["age", "grade_level", "math_score", "science_score", "english_score", "history_score"]:
                    row[key] = int(row[key])

                if key == "attendance_rate":
                    row["attendance_rate"] = float(row["attendance_rate"]) * 100

            students.append(row)

    return students

### Justification for `load_student_data(filename)` Method

The `load_student_data(filename)` method is used to load the dataset from the csv file, and then the appropriate values are transformed from string to int and float, to ensure the dataset is ready for analysis.

In [200]:
def map_score_to_grade_letter(score):
    """
    Converts a given score to a valid grade letter
    """

    if score < 0 or score > 100:
        raise Exception("The provided score must be a value between 0 and 100")
    
    grade = "F"
    
    if score >= 90:
        grade = "A"
    elif score >= 80:
        grade = "B"
    elif score >= 70:
        grade = "C"
    elif score >= 60:
        grade = "D"
    elif score >= 50:
        grade = "E"

    return grade

In [201]:
def calc_total_score_for_students(students):
    """
    Calculates the total score for each student record, and introduces a new "total_score" key to the student record
    """
    for student in students:
        total_score = ((student["math_score"] + student["english_score"] + student["science_score"] + student["history_score"])) / 4
        student["total_score"] = total_score
        student["grade"] = map_score_to_grade_letter(total_score)

    return students

In [202]:
def calculate_statistics(students):
    """
    Calculates statistical measures for students' total score"
    """
    scores = []
    for student in students:
        scores.append(student["total_score"])

    # An alternative way to achieve the above using list comprehension
    # scores = [student["total_score"] for student in students]

    return {
        "mean": statistics.mean(scores),
        "median": statistics.median(scores),
        # Ternary operator: a shorthand way of writing a simple if/else statement
        "std_dev": statistics.stdev(scores) if len(scores) > 1 else 0,
        "min": min(scores),
        "max": max(scores),
    }

In [203]:
def get_student_total_score(student):
    return student["total_score"]

def get_best_performing_student(students):
    """
    Returns the student with the highest total score
    """
    student = max(students, key=get_student_total_score)

    return f"{student["first_name"]} {student["last_name"]}"

In [204]:
def generate_school_report(students):
    """
    Compile students' performance report and output to a text file.
    """
    best_student = get_best_performing_student(students)
    stats = calculate_statistics(students)

    with open(REPORT_FILE, "w") as file:
        file.write("SCHOOL REPORT\n")
        file.write("="*60 + "\n\n\n")

        # Stats
        file.write("SCHOOL STATISTICS\n")
        file.write("-"*60 + "\n")
        
        file.write(f"Total Number of Students: {len(students)}\n")
        file.write(f"Score Mean: {stats["mean"]:.1f}\n")
        file.write(f"Score Median: {stats["median"]:.1f}\n")
        file.write(f"Score Standard Deviation: {stats["std_dev"]:.1f}\n")
        file.write(f"Score Range: From {stats["min"]} to {stats["max"]}\n")


        file.write("\n\nSTUDENT RECOGNITION\n")
        file.write("-"*60 + "\n")

        file.write(f"Best Performing Student: {best_student}\n")


        file.write("\n\nSTUDENTS' DETAILS HIGHLIGHT\n")
        file.write("-"*60 + "\n")

        file.write(f"{'Name':<24} {'Score':<8} {'Attendance':<12} {'Grade':<5}\n")
        file.write("-"*60 + "\n")

        for student in sorted(students, key=get_student_total_score, reverse=True):
            student_name = f"{student["first_name"]} {student["last_name"]}" 

            file.write(f"{student_name:<24} {student["total_score"]:<8} {student["attendance_rate"]:<12} {student["grade"]:<5}\n")
            

In [205]:
dataset = load_student_data(STUDENTS_DATA_FILE)
students = calc_total_score_for_students(dataset)

generate_school_report(students)